In [ ]:
# %load extract_img_feats.py
import os
import numpy as np
from skimage import io
import torch
from torch.nn import *
import torchvision.models as models

# image model
image_model = models.resnet152(pretrained=True).cuda()
modules = list(image_model.children())[:-1]
image_model = Sequential(*modules)
image_model.eval()

root_dir = 'small_dataset_objects'

for split in ['train', 'test', 'val']:
    csv_file = os.path.join(root_dir, split + '_split.tsv')
    file_list = []
    for line in open(csv_file, 'r').readlines()[1:]:
        file_list.append(line.split('\t'))
    
    out_path = os.path.join(root_dir, split, 'img_feats')
    
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    
    cnt = 0
    for line in file_list:
        cnt += 1
        print(cnt)
        img_name = os.path.join(root_dir, split, 'img', line[0]+'.jpg')
        image = torch.from_numpy(np.rollaxis(io.imread(img_name), 2, 0)).unsqueeze(0).float().cuda()
        feats = image_model(image).view(-1)
        out_file = os.path.join(out_path, line[0])
        np.save(out_file, feats.cpu().detach().numpy()) 
